In [1]:
import json
import math

from collections import Counter

import nltk

from watson_text_talker import *

import speech_recognition as sr

pygame 2.0.0 (SDL 2.0.12, python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
with open('../env.json') as env:
    env = json.load(env)

In [3]:
config = TT_Config()
config.API_KEY = env['API_KEY']
config.TTS_VOICE = 'en-US_MichaelVoice'
config.API_URL = env['API_URL']

In [4]:
text_talker = TextTalker(config=config)

In [5]:
text_talker.say('Hello World!  I finally have a voice!')

True

In [6]:
def record_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print('Say something!')
        audio = r.listen(source)

    try:
        data = r.recognize_google(audio)
    except:
        data = 'asdfjkl;'
    return data

In [7]:
while True:
    text = record_audio()
    res = ''
    
    if 'are you there' in text:
        text_talker.say('Yes, I have successfully initialized and am ready to learn!')

Say something!


KeyboardInterrupt: 

In [8]:
def consine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[word] * vec2[word] for word in intersection])
    
    sum1 = sum([vec1[word] ** 2 for word in list(vec1.keys())])
    sum2 = sum([vec2[word] ** 2 for word in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [9]:
def vectorize(sent):
    lemma = nltk.stem.WordNetLemmatizer()
    words = [lemma.lemmatize(word.lower()) for word in nltk.word_tokenize(sent)]
    return Counter(words)

In [10]:
def compare(sent1, sent2):
    vec1 = vectorize(sent1)
    vec2 = vectorize(sent2)
    
    return consine_similarity(vec1, vec2)

In [11]:
cats = ['cat', 'i love my cat']
dogs = ['dog', 'i love my dog']
birds = ['bird', 'i love my bird']
other = ['house', 'i love my house']

topics = {
    'cats': cats,
    'dogs': dogs,
    'birds': birds,
    'other': other
}

In [12]:
def classify(sent):
    best_match = ('other', 0.0)
    for topic, values in topics.items():
        comparisons = [compare(sent, value) for value in values]
        avg_comp = sum(comparisons) / len(comparisons)
        if avg_comp >= best_match[1]:
            best_match = (topic, avg_comp)
    return best_match

In [36]:
def correct(best_match, sent):
    text_talker.say(f'Are you talking about this topic: {best_match[0]}?')
    res = record_audio()
    print(res)
    #res = input(f'Are you talking about this topic: {best_match[0]}?\n')
    print(res)
    if res == 'yes':
        topics[best_match[0]].append(sent)
    else:
        text_talker('Okay, what should the topic be?')
        res = record_audio()
        print(res)
        #res = input('Okay, what should the topic be?\n')
        topics[res].append(sent)
    text_talker.say('Great, I will remember that')
    print('Great, I will remember that')

In [14]:
def learn(sent):
    best_match = classify(sent)
    correct(best_match, sent)

In [37]:
while True:
    text = record_audio()
    text = text.lower()
    
    if 'hello robot' in text:
        text_talker.say('Hi!')
        text = text.replace('hello robot','')
        
        if 'what am i saying' in text:
            text = text.replace('what am i saying','')
            learn(text)

Say something!
Say something!
yes
yes
Great, I will remember that
Say something!
Say something!
yes
yes
Great, I will remember that
Say something!


KeyboardInterrupt: 

In [15]:
learn('I love my new apartment')
topics

Are you talking about this topic: other?
 yes


Great, I will remember that


{'cats': ['cat', 'i love my cat'],
 'dogs': ['dog', 'i love my dog'],
 'birds': ['bird', 'i love my bird'],
 'other': ['house', 'i love my house', 'I love my new apartment']}